In [1]:
from calculate_reaction_rates import *
from sigmav_functions import *
from power import *
from units_and_constants import *
import itertools
import pprint
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from scipy.stats import sobol_indices, uniform

Calculate the Sobol Indices for the following quantities:

| Quantity           | Unit  |   | Quantity         | Unit  |
|--------------------|-------|---|------------------|-------|
| Tdot_fusion        | 1/s   |   | Pf_DD            | MW    |
| Tdot_breedingDT    | 1/s   |   | Pf_DD_DT         | MW    |
| Tdot_breedingDD    | 1/s   |   | Pf_DD_DHe3       | MW    |
| Tdot_diff          | 1/s   |   | Pf_DD_tot        | MW    |
| Tdot_tot           | 1/s   |   | Pf_DT            | MW    |
| startup_time       | hour  |   | P_e_net_DD       | MW    |
| Q_DD               | -     |   | P_e_net_DT       | MW    |
| Q_DT               | -     |   | E_lost           | MJ    |
| Dollar_lost        | USD   |   |                  |       |

Varying these parameters in the following ranges:

| Parameter         | Unit     | Range                |     | Parameter     | Unit     | Range                |
|-------------------|----------|----------------------|-----|--------------|----------|----------------------|
| V_plasma          | m³       | [1.00e+02,2.00e+03]  |     | TBR          | -        | [5.00e-01,1.30e+00]  |
| tau_p_T           | s        | [1.00e-03,5.00e+00]  |     | TBR_DDn      | -        | [5.00e-01,1.00e+00]  |
| tau_p_He3         | s        | [1.00e-03,5.00e+00]  |     | P_aux        | MW       | [1.00e-01,1.00e+00]  |
| n_e_avg           | m⁻³      | [1.00e+20,1.00e+21]  |     | eta_th       | -        | [1.00e-01,5.00e-01]  |
| T_e_avg           | keV      | [1.00e+01,5.00e+01]  |     | Cost_per_kWh | USD/kWh  | [1.50e-01,5.00e-01]  |
| I_ST              | kg       | [1.00e+00,5.00e+00]  |     |              |          |                      |


# Ranges:

In [ ]:
dists = [
    uniform(loc=100, scale=2000-50),        # V_plasma (up to about twice of ITER volume - DEMO-like?)
    uniform(loc=1e-3, scale=5-1e-3),        # tau_p_T
    uniform(loc=1e-3, scale=5-1e-3),        # tau_p_He3
    uniform(loc=1e20, scale=1e21-1e20),     # n_e_avg
    uniform(loc=10, scale=50-10),           # T_e_avg
    uniform(loc=1, scale=5-1),              # I_ST
    uniform(loc=0.5, scale=1.3-0.5),        # TBR
    uniform(loc=0.5, scale=1.0-0.5),        # TBR_DDn
    uniform(loc=0.1, scale=1.0-0.1),        # P_aux
    uniform(loc=0.1, scale=0.5-0.1),        # eta_th
    uniform(loc=0.15, scale=0.5-0.15),      # Cost_per_kWh
]

In [ ]:
def model_func(x):
    # x shape: (d, n)
    # Unpack parameters
    V_PLASMA, TAU_P_T, TAU_P_HE3, N_E_AVG, T_E_AVG, I_ST, TBR_DT, TBR_DDN, P_AUX, ETA_TH, COST_PER_KWH = x
    results = []
    for i in range(x.shape[1]):
        # Convert floats to Pint quantities
        V_plasma = V_PLASMA[i] * unit.m**3
        tau_p_T = TAU_P_T[i] * unit.s
        tau_p_He3 = TAU_P_HE3[i] * unit.s
        n_e_avg = N_E_AVG[i] * unit.m**-3
        T_e_avg = T_E_AVG[i] * unit.keV
        I_st = I_ST[i] * unit.kg
        TBR = TBR_DT[i]
        TBR_DDn = TBR_DDN[i]
        P_aux = P_AUX[i] * unit.MW
        eta_th = ETA_TH[i]
        Cost_per_kwh = COST_PER_KWH[i] / unit('kWh')
        n_D = n_e_avg
        # Calculate the reaction rates
        DD_reaction_rates = calculate_reaction_rates_DD(n_D, T_e_avg, V_plasma, tau_p_T, tau_p_He3)
        # ESTIMATE TRITIUM PRODUCTION
        # calculate the tritium production rates
        Tdot_fusion = DD_reaction_rates["R_DDp"] - DD_reaction_rates["R_DT"] # [1/s] rate of tritium production due to DDp fusions, considering the losses due to DT neutrons (NB. It is assumed that all the Tritium that is not burnt in DT fusios can be extracted from the system)
        Tdot_breedingDT = TBR * DD_reaction_rates["R_DT"] # [1/s] is the rate of tritium production due to DT neutrons interacting with the breeding blanket
        Tdot_breedingDD = TBR_DDn * DD_reaction_rates["R_DDn"] # [1/s] is the rate of tritium production due to DD neutrons interacting with the Li6 in the breeding blanket
        Tdot_diff = V_plasma * DD_reaction_rates["density_T"] / tau_p_T # [1/s] is the rate of tritium production due to diffusion of tritium in the breeding blanket
        Tdot_tot = Tdot_fusion + Tdot_breedingDT + Tdot_breedingDD # [1/s] is the total rate of tritium production in the system
        # CALCULATE THE STARTUP TIME
        N_ST = I_st/molecular_weight_T.to("kg/mol")*N_A # [-] is the number of tritium atoms needed for startup (I_ST[g]/3.016[g/mol]*6.022e23[atoms/mol])
        startup_time = N_ST / (Tdot_tot) # [s] is the time needed to produce the required amount of tritium for startup
        # CALCULATE THE FUSION POWER
        Pf_DD = DD_reaction_rates["R_DDp"]*E_DDp + DD_reaction_rates["R_DDn"]*E_DDn # [W] is the power produced by DD reactions
        Pf_DD_DT = DD_reaction_rates["R_DT"]*E_DT # [W] is the power produced by DT sub-reactions
        Pf_DD_DHe3 = DD_reaction_rates["R_DHe3"]*E_DHe3 # [W] is the power produced by DHe3 sub-reactions
        Pf_DD_tot = Pf_DD.to('MW') + Pf_DD_DT.to('MW') + Pf_DD_DHe3.to('MW') # [W] is the total power produced in a DD reactor
        Pf_DT = fusion_power_50D50T(n_e_avg, T_e_avg, E_DT, V_plasma) # [W] is the power produced by DT reactions (the 1e-6 factor is needed to convert <sigmav> from cm^3/s to m^3/s)
        # CALCULATE THE NET ELECTRICAL POWER
        
        P_e_net_DD, Q_DD = calculate_P_e_net_Paux(Pf_DD, P_aux, eta_th) # [W] is the net electrical power produced by the reactor
        P_e_net_DT, Q_DT = calculate_P_e_net_Paux(Pf_DT, P_aux, eta_th) # [W] is the net electrical power produced by the reactor

        E_lost = np.abs(P_e_net_DD*startup_time - P_e_net_DT*startup_time) # [J] is the energy lost during the startup time operation in DD
        Dollar_lost = Cost_per_kwh.to('1/J') * E_lost # [USD] is the cost of the lost energy during the startup time operation in DD
        # Collect all values as magnitudes (convert units if needed)
        row = [
            Tdot_fusion.magnitude, 
            Tdot_breedingDT.magnitude, 
            Tdot_breedingDD.magnitude, 
            Tdot_diff.magnitude, 
            Tdot_tot.magnitude,
            startup_time.to('hour').magnitude,  # or .magnitude for seconds
            Pf_DD.to('MW').magnitude, 
            Pf_DD_DT.to('MW').magnitude, 
            Pf_DD_DHe3.to('MW').magnitude, 
            Pf_DD_tot.magnitude,
            Pf_DT.to('MW').magnitude,
            P_e_net_DD.to('MW').magnitude, 
            Q_DD, 
            P_e_net_DT.to('MW').magnitude, 
            Q_DT,
            E_lost.to('MJ').magnitude, 
            Dollar_lost.magnitude
        ]
        results.append(row)
    return np.array(results).T

# Sobol Indices

In [4]:
d = len(dists)
n = 2**12 # Must be a power of 2; increase for more accuracy

# sobol_indices will generate the samples and call your function
sobol = sobol_indices(
    func=model_func,
    n=n,
    dists=dists,
    method='saltelli_2010'
)

In [8]:
#threshold to highlight most important indices
threshold = 0.2

# Sobol Indices fo $_lost

In [12]:
Dollar_lost_index = -1

In [13]:
cols = [
    "V_plasma", "tau_p_T", "tau_p_He3", "n_e_avg", "T_e_avg", "I_ST",
    "TBR", "TBR_DDn", "P_aux", "eta_th", "Cost_per_kWh"
]

col_width = 20

# Prepare the ranges
ranges = []
for dist in dists:
    lower = dist.kwds['loc']
    upper = dist.kwds['loc'] + dist.kwds['scale']
    ranges.append(f"[{lower:.2e},{upper:.2e}]".ljust(col_width))

# Calculate sums
first_order = sobol.first_order[Dollar_lost_index, :]
total_order = sobol.total_order[Dollar_lost_index, :]
sum_indices = first_order + total_order

# Print header row (add SUM column)
print("PARAMETER".ljust(col_width) + "".join([name.upper().ljust(col_width) for name in cols]) + "SUM".ljust(col_width))
# Print ranges row
print("RANGE".ljust(col_width) + "".join(ranges) + "".ljust(col_width))
# Print first-order indices row
print("FIRST-ORDER".ljust(col_width) + "".join([("%.4f" % v).ljust(col_width) for v in first_order]) + "%.4f".ljust(col_width) % np.sum(first_order))
# Print total-order indices row
print("TOTAL-ORDER".ljust(col_width) + "".join([("%.4f" % v).ljust(col_width) for v in total_order]) + "%.4f".ljust(col_width) % np.sum(total_order))

print(f"\nParameters with first-order or total-order index > {threshold}:")
for i in range(len(sobol.first_order[Dollar_lost_index])):
    first = sobol.first_order[Dollar_lost_index, i]
    total = sobol.total_order[Dollar_lost_index, i]
    print_name = cols[i] if i < len(cols) else f"param_{i}"
    if first > threshold or total > threshold:
        dist = dists[i]
        lower = dist.kwds['loc']
        upper = dist.kwds['loc'] + dist.kwds['scale']
        print(f"    {print_name}: first-order = {first:.2f}, total-order = {total:.2f}, range = [{lower:.2e}, {upper:.2e}]")

PARAMETER           V_PLASMA            TAU_P_T             TAU_P_HE3           N_E_AVG             T_E_AVG             I_ST                TBR                 TBR_DDN             P_AUX               ETA_TH              COST_PER_KWH        SUM                 
RANGE               [1.00e+02,2.05e+03] [1.00e-03,5.00e+00] [1.00e-03,5.00e+00] [1.00e+20,1.00e+21] [1.00e+01,5.00e+01] [1.00e+00,5.00e+00] [5.00e-01,1.30e+00] [5.00e-01,1.00e+00] [1.00e-01,1.00e+00] [1.00e-01,5.00e-01] [1.50e-01,5.00e-01]                     
FIRST-ORDER         -0.0000             0.0005              0.0000              0.0006              0.1244              0.2657              0.0044              0.0147              -0.0000             0.2625              0.1720              0.8448                
TOTAL-ORDER         0.0000              0.0015              0.0000              0.0008              0.1831              0.3569              0.0078              0.0222              0.0000              0.3603         

# Sobol Indices fo t_startup

In [14]:
t_startup_index = 5  # Index for startup time in the results

In [15]:
cols = [
    "V_plasma", "tau_p_T", "tau_p_He3", "n_e_avg", "T_e_avg", "I_ST",
    "TBR", "TBR_DDn", "P_aux", "eta_th", "Cost_per_kWh"
]

col_width = 20

# Prepare the ranges
ranges = []
for dist in dists:
    lower = dist.kwds['loc']
    upper = dist.kwds['loc'] + dist.kwds['scale']
    ranges.append(f"[{lower:.2e},{upper:.2e}]".ljust(col_width))

# Calculate sums
first_order = sobol.first_order[t_startup_index, :]
total_order = sobol.total_order[t_startup_index, :]
sum_indices = first_order + total_order

# Print header row (add SUM column)
print("PARAMETER".ljust(col_width) + "".join([name.upper().ljust(col_width) for name in cols]) + "SUM".ljust(col_width))
# Print ranges row
print("RANGE".ljust(col_width) + "".join(ranges) + "".ljust(col_width))
# Print first-order indices row
print("FIRST-ORDER".ljust(col_width) + "".join([("%.4f" % v).ljust(col_width) for v in first_order]) + "%.4f".ljust(col_width) % np.sum(first_order))
# Print total-order indices row
print("TOTAL-ORDER".ljust(col_width) + "".join([("%.4f" % v).ljust(col_width) for v in total_order]) + "%.4f".ljust(col_width) % np.sum(total_order))

print(f"\nParameters with first-order or total-order index > {threshold}:")
for i in range(len(sobol.first_order[t_startup_index])):
    first = sobol.first_order[t_startup_index, i]
    total = sobol.total_order[t_startup_index, i]
    print_name = cols[i] if i < len(cols) else f"param_{i}"
    if first > threshold or total > threshold:
        dist = dists[i]
        lower = dist.kwds['loc']
        upper = dist.kwds['loc'] + dist.kwds['scale']
        print(f"    {print_name}: first-order = {first:.2f}, total-order = {total:.2f}, range = [{lower:.2e}, {upper:.2e}]")

PARAMETER           V_PLASMA            TAU_P_T             TAU_P_HE3           N_E_AVG             T_E_AVG             I_ST                TBR                 TBR_DDN             P_AUX               ETA_TH              COST_PER_KWH        SUM                 
RANGE               [1.00e+02,2.05e+03] [1.00e-03,5.00e+00] [1.00e-03,5.00e+00] [1.00e+20,1.00e+21] [1.00e+01,5.00e+01] [1.00e+00,5.00e+00] [5.00e-01,1.30e+00] [5.00e-01,1.00e+00] [1.00e-01,1.00e+00] [1.00e-01,5.00e-01] [1.50e-01,5.00e-01]                     
FIRST-ORDER         0.0241              -0.0002             0.0000              0.0931              0.0405              0.0027              -0.0001             0.0007              0.0000              0.0000              0.0000              0.1609                
TOTAL-ORDER         0.2825              0.0000              0.0000              0.4553              0.2217              0.0633              0.0002              0.0041              0.0000              0.0000         